In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("wrangling") \
    .getOrCreate()

In [2]:
df = spark.read.json("/common/users/ac1771/realnews/realnews.jsonl")

In [3]:
df.show()

+--------------------+------------------+------------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|             authors|            domain|publish_date|split|         status|             summary|                text|               title|                 url|            url_used|     warc_date|
+--------------------+------------------+------------+-----+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+
|       [Dairy Queen]|      mashable.com|  09-21-2016|train|success_wayback|Pretzel sticks, p...|You never really ...|Here's what happe...|http://mashable.c...|https://web.archi...|    2016092119|
|[Jessica Stahl, A...|washingtonpost.com|  12-09-2013|train|success_wayback|We Catholics have...|The message of th...|In Pope Francis e...|http://www.washin...|https://web.archi...|    2013121219|
|   [Tobias Sal

In [10]:
df = df.drop("split", "status", "text", "url", "url_used", "warc_date")
df = df.na.drop(subset=["summary"])

+--------------------+------------------+------------+--------------------+--------------------+
|             authors|            domain|publish_date|             summary|               title|
+--------------------+------------------+------------+--------------------+--------------------+
|       [Dairy Queen]|      mashable.com|  09-21-2016|Pretzel sticks, p...|Here's what happe...|
|[Jessica Stahl, A...|washingtonpost.com|  12-09-2013|We Catholics have...|In Pope Francis e...|
|   [Tobias Salinger]|   nydailynews.com|  07-22-2015|A woman who alleg...|Aussie who killed...|
|        [Gary Myers]|   nydailynews.com|  10-04-2015|It’s a huge bonus...|Myers: Jets, Gian...|
|    [Louis Columbus]|        forbes.com|  01-18-2014|The five highest ...|Best- And Worst-P...|
|      [Chris Powers]|   nydailynews.com|  03-30-2016|If Vegas had hand...|Volvo station wag...|
|    [Chris Cillizza]|washingtonpost.com|  01-16-2014|Ed Gillespie is t...|How Senate Republ...|
|  [Patrick Kingsley]|   thegu

In [14]:
df.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- domain: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- title: string (nullable = true)



In [17]:
df.write.mode("Overwrite").json('/common/users/shared/cs543_fall22_group3/realnews/realnews_truncated')